In [1]:
import json
import pandas as pd
from pymongo import MongoClient
import matplotlib.pyplot as plt
import scipy as sp
import scipy.stats as stats
from helpers import plotOwners, plotNewOwners, remove0s

client = MongoClient('localhost', 27017)
db = client.steamspy

#creating new columns to be used for data and casting things as ints
data = pd.DataFrame(list(db.clean.find()))
data.drop(columns = "_id", inplace = True) #drop _id column
#making price column ints and making price of free games 0
data['price'] = data['price'].fillna(0)
data['price'] = data['price'].astype(int)
data['negative'] = data['negative'].astype(int)
data['positive'] = data['positive'].astype(int)
data['Like/Dislike Ratio'] = data['positive']/data['negative']
data = data[data['Genres'].apply(lambda x: 'Indie' not in x)]
data = data[data.price != 0]
data['Owner Average'] = (data['llOwners'] + data['ulOwners'])/2
data['earnings'] = data['price'] * data['Owner Average']
data = remove0s(data)

In [2]:
def hasGenre(series, listOfGenres):
    for genre in listOfGenres:
        if genre in series:
            return True
    return False

def notGenre(series, listOfGenres):
    for genre in listOfGenres:
        if genre in series:
            return False
    return True

In [3]:
action = data[data['Genres'].apply(lambda x: hasGenre(x, ['Action']))]
otherAction = data[data['Genres'].apply(lambda x: notGenre(x, ['Action']))]
mmo = data[data['Genres'].apply(lambda x: hasGenre(x, ['MMO']))]
otherMMO = data[data['Genres'].apply(lambda x: notGenre(x, ['MMO']))]
adventure = data[data['Genres'].apply(lambda x: hasGenre(x, ['Adventure']))]
otherAdventure = data[data['Genres'].apply(lambda x: notGenre(x, ['Adventure']))]
strategy = data[data['Genres'].apply(lambda x: hasGenre(x, ['Strategy']))]
otherStrategy = data[data['Genres'].apply(lambda x: notGenre(x, ['Strategy']))]

In [4]:
#p-value action vs other
res = stats.mannwhitneyu(action['earnings'], otherAction['earnings'], alternative="greater")
print(res.pvalue)

7.724700016751446e-41


In [5]:
#p-value mmo vs other
res = stats.mannwhitneyu(mmo['earnings'], otherMMO['earnings'], alternative="greater")
print(res.pvalue)

0.0014406562278334143


In [6]:
#p-value adventure vs other
res = stats.mannwhitneyu(adventure['earnings'], otherAdventure['earnings'], alternative="greater")
print(res.pvalue)

0.9999975305970101


In [7]:
#p-value strategy vs other
res = stats.mannwhitneyu(strategy['earnings'], otherStrategy['earnings'], alternative="greater")
print(res.pvalue)

0.13564290780299637


In [9]:
#p-value mmo vs action
res = stats.mannwhitneyu(mmo['earnings'], action['earnings'], alternative="greater")
print(res.pvalue)

0.0917966448853122
